In [0]:
folder = '/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/'

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

Using TensorFlow backend.


In [0]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import pandas as pd
import wave
import sys
import os
import librosa
import librosa.display
import xgboost as xgb
from  sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV 
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import sklearn.naive_bayes as nb
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier,GradientBoostingClassifier
from sklearn.neighbors import kd_tree
import seaborn as sn
from sklearn.metrics import confusion_matrix
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint


from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

In [0]:
# Rad the Saved Features
filename = folder + 'RC-lpc.csv'
df = pd.read_csv(filename,header=None)

In [15]:
df.shape

(6898, 3)

In [0]:
# Load per cycle data frame
# This only loaded to get the corresponding labels
import pickle
folder = "/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/" 
filename = folder + 'w_c_dataset.pickle'
infile = open(filename,'rb')
[sound,sr,lengths,times,labels] = pickle.load(infile)
infile.close()

In [0]:
def split_sounds(sounds,times,labels):
    s=[]
    l=[]
    for i,sound in enumerate(sounds):
        for t,label in zip(times[i],labels[i]):
            s.append(sound[int(t[0]):int(t[1])])
            if label==0:
                a=np.array([1,0,0,0])
            if label==1:
                a=np.array([0,1,0,0])
            if label==2:
                a=np.array([0,0,1,0])    
            if label==3:
                a=np.array([0,0,0,1])
            l.append(a)
    return s,l

In [0]:
[data,label]=split_sounds(sound,times,labels)

In [19]:
# add channel dimension for CNN
dataset = df

label=np.asarray(label)
a=np.zeros(label.shape[0])
for i in range(label.shape[0]):
    for j in range(label.shape[1]):
        if label[i][j]==1:
            a[i]=j

# COnvert to binary for the binary classification problem
bin_label =np.zeros(label.shape[0])
for index,l in enumerate(a):
    if l == 0:
        bin_label[index] = l
    else:
        bin_label[index] = 1

unique, counts = np.unique(bin_label, return_counts=True)

print(np.asarray((unique, counts)).T)


# One-hot encode labels
le = LabelEncoder()
i_labels = le.fit_transform(bin_label)
oh_labels = to_categorical(i_labels)

[[0.000e+00 3.642e+03]
 [1.000e+00 3.256e+03]]


In [0]:
scaler=StandardScaler()
x_train,x_test,y_train,y_test = sklearn.model_selection.train_test_split(df,oh_labels,test_size=0.3, random_state=42,stratify=i_labels)
scaler.fit(x_train)
x_train=scaler.transform(x_train)
x_test=scaler.transform(x_test)

In [23]:
# Custom model for classification
model = Sequential()

model.add(Dense(3, activation='relu', input_shape=(3,),kernel_initializer='random_normal'))

#model.add(Dense(200, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(300, activation='relu'))

#model.add(Dense(150, activation='relu')) 
#model.add(Dropout(0.2))
#model.add(Dense(75, activation='relu')) 
model.add(Dropout(0.2))
#model.add(Dense(64, activation='relu',kernel_initializer='random_normal')) 
#model.add(Dropout(0.2))


#model.add(Dense(16, activation='relu') )

model.add(Dense(2, activation='sigmoid')) 

# Compile the model
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam') 

model.summary()



# Load the trained model if required
#model_file = '/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/LogMel_MFCC_Chroma_CNN/Try6_Unlinked/model_40_230.h5'
#model.load_weights(model_file)

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=1)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 3)                 12        
_________________________________________________________________
dropout_1 (Dropout)          (None, 3)                 0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8         
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________
65/65 [==============================] - 0s 1ms/step - loss: 0.6942 - accuracy: 0.4715
Pre-training accuracy: 47.1498%


In [24]:
#train model
num_epochs = 1000
num_batch_size = 128



callbacks = [
    ModelCheckpoint(
        filepath='/content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/FinalModels/LPC_ANN_Binary/LPC_ANN_Binary_{epoch:02d}.h5',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs,
          validation_data=(x_test, y_test), callbacks=callbacks, verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/1000
 1/38 [..............................] - ETA: 0s - loss: 0.6953 - accuracy: 0.4453
Epoch 00001: val_accuracy improved from -inf to 0.53140, saving model to /content/drive/My Drive/Respiratory_Sound_Database/Respiratory_Sound_Database/FinalModels/LPC_ANN_Binary/LPC_ANN_Binary_01.h5
38/38 [==============================] - 0s 3ms/step - loss: 0.6934 - accuracy: 0.5014 - val_loss: 0.6927 - val_accuracy: 0.5314
Epoch 2/1000
 1/38 [..............................] - ETA: 0s - loss: 0.6928 - accuracy: 0.5312
Epoch 00002: val_accuracy did not improve from 0.53140
38/38 [==============================] - 0s 2ms/step - loss: 0.6924 - accuracy: 0.5280 - val_loss: 0.6924 - val_accuracy: 0.5280
Epoch 3/1000
 1/38 [..............................] - ETA: 0s - loss: 0.6923 - accuracy: 0.5391
Epoch 00003: val_accuracy did not improve from 0.53140
38/38 [==============================] - 0s 2ms/step - loss: 0.6922 - accuracy: 0.5280 - val_loss: 0.6922 - val_accuracy: 0.5280
Epoch 4/1000
 1/